# Video Message Generator from Youtube


Hello everyone, today we are going to build an interesting application that creates video messages from Youtube Videos.
The application that we will create will be on hosted on **Amazon Web Services (AWS)** , in ordering to to perform the calculations

We are going to use SageMaker Notebook to create this application.

There are two programs to run this app.
- The notebook version (.ipynb ) that is the extended version that we are going to see first,
- The python version  (.py) that is the simplified version.

First we are going to discuss the extended version to understand how the things were done, and the simplifed version at the end.


# Step 1 - Cloning the repository
The first step  is clone the repository, open the terminal in SageMaker and type the following

```
git clone https://github.com/ruslanmv/Video-Message-Generator.git
```
then enter to the directory
```
cd Video-Message-Generator

```

# Step 2 - Enviroment setup
First at all, we need to install the environment to this application, that will be **VideoMessage**, that will be executed on **python=3.7.13** and it is required
**ffmpeg** and **git-lfs**.

To do this task. open a terminal and type:
```
sh-4.2$ sh install.sh
```
after is done. You can open the  **Video-Message-Generator.ipynb** notebook and choose the kernell **VideoMessage**.

## Step 3 - Definition of variables.
In odering to begin the construction of the application we requiere to manage well the enviroments of this Cloud Service.
In SageMaker Notebook 

In [7]:
import sys ,os
print(sys.prefix)

/home/ec2-user/anaconda3/envs/VideoMessage


However if we want to install to our enviroment by using our terminal we have to be careful, becase Sagemaker runs on their own container,
then in the  terminal of Sagemaker we have to load propertly the enviroment that we have created

In [8]:
Sagemaker = True
if Sagemaker :
    env='source activate python3 && conda activate VideoMessage &&'
else:
    env='' 

## Step 4. Setup of the dependencies

In this part we have install all dependencies of our custom program
Due to we are going to install for first time, we define True otherwise False

In [9]:
is_first_time = True

In [10]:
#Install dependency
# Download pretrained model
# Import the os module
import os
# Get the current working directory
parent_dir = os.getcwd()
print(parent_dir)
if is_first_time:
    # Directory 
    directory = "sample_data"
    # Path 
    path = os.path.join(parent_dir, directory) 
    print(path)
    try:
        os.mkdir(path)
        print("Directory '% s' created" % directory) 
    except Exception:
         print("Directory '% s'was already created" % directory)
    os.system('git clone https://github.com/Rudrabha/Wav2Lip')
    os.system('cd Wav2Lip &&{} pip install  -r requirements.txt'.format(env))

/home/ec2-user/SageMaker/VideoMessageGen
/home/ec2-user/SageMaker/VideoMessageGen/sample_data
Directory 'sample_data'was already created


fatal: destination path 'Wav2Lip' already exists and is not an empty directory.


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 67.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with sta

In [13]:
from utils.default_models import ensure_default_models
from pathlib import Path

In [14]:
## Load the models one by one.
print("Preparing the models of Wav2Lip")
ensure_default_models(Path("Wav2Lip"))

Preparing the models of Wav2Lip
Wav2Lip/checkpoints/wav2lip_gan.pth
Wav2Lip/face_detection/detection/sfd/s3fd.pth


In [15]:
if is_first_time:
    os.system('git clone https://github.com/Edresson/Coqui-TTS -b multilingual-torchaudio-SE TTS')
    os.system('{} pip install -q -e TTS/'.format(env))
    os.system('{} pip install -q torchaudio==0.9.0'.format(env))
    os.system('{} pip install -q youtube-dl'.format(env))
    os.system('{} pip install ffmpeg-python'.format(env))
    os.system('{} pip install gradio==3.0.4'.format(env))
    os.system('{} pip install pytube==12.1.0'.format(env))

fatal: destination path 'TTS' already exists and is not an empty directory.


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [23]:
#this code for recording audio
from IPython.display import HTML, Audio
#from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from pytube import YouTube
import random
from subprocess import call
import os
from IPython.display import HTML
from base64 import b64encode
from IPython.display import clear_output
from datetime import datetime

In [24]:
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

In [33]:
def time_between(t1, t2):
    FMT = '%H:%M:%S'
    t1 = datetime.strptime(t1, FMT)
    t2 = datetime.strptime(t2, FMT)
    delta = t2 - t1
    return str(delta)

In [34]:
time_between("00:00:01","00:00:10" )

'0:00:09'

In [35]:
def download_video(url):
    print("Downloading...")
    local_file = (
        YouTube(url)
        .streams.filter(progressive=True, file_extension="mp4")
        .first()
        .download(filename="youtube{}.mp4".format(random.randint(0, 10000)))
    )
    print("Downloaded")
    return local_file
  # download(output_path=destination, filename="name.mp4")

In [36]:
def download_youtube(url):
    #Select a Youtube Video
    #find youtube video id
    from urllib import parse as urlparse
    url_data = urlparse.urlparse(url)
    query = urlparse.parse_qs(url_data.query)
    YOUTUBE_ID = query["v"][0]
    url_download ="https://www.youtube.com/watch?v={}".format(YOUTUBE_ID)
    # download the youtube with the given ID
    #call(["youtube-dl", "-f","bestvideo[ext=mp4]","--output","youtube.%(ext)s",url_download])
    #call(["youtube-dl", "-f","mp4","--output","youtube.%(ext)s",url_download])
    os.system("{} youtube-dl -f  mp4 --output youtube.mp4 '{}'".format(env,url_download))
    #!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

In [37]:
if is_first_time:
    URL = 'https://www.youtube.com/watch?v=xw5dvItD5zY'
    #URL = 'https://www.youtube.com/watch?v=uIaY0l5qV0c'
    #download_youtube(URL)
    download_video(URL)

Downloading...
Downloaded


In [38]:
def cleanup():
    import pathlib
    import glob
    types = ('*.mp4','*.mp3', '*.wav') # the tuple of file types
    #Finding mp4 and wave files
    junks = []
    for files in types:
        junks.extend(glob.glob(files))
    try:    
        # Deleting those files
        for junk in junks:
            print("Deleting",junk)
            # Setting the path for the file to delete
            file = pathlib.Path(junk)
            # Calling the unlink method on the path
            file.unlink()               
    except Exception:
        print("I cannot delete the file because it is being used by another process")

In [39]:
def clean_data():
    # importing all necessary libraries
    import sys, os
    # initial directory
    home_dir = os.getcwd()
    # some non existing directory
    fd = 'sample_data/'
    # Join various path components
    path_to_clean=os.path.join(home_dir,fd)
    print("Path to clean:",path_to_clean)
    # trying to insert to false directory
    try:
        os.chdir(path_to_clean)
        print("Inside to clean", os.getcwd())
        cleanup()
    # Caching the exception
    except:
        print("Something wrong with specified\
        directory. Exception- ", sys.exc_info())
    # handling with finally
    finally:
        print("Restoring the path")
        os.chdir(home_dir)
        print("Current directory is-", os.getcwd())

In [40]:
def youtube_trim(url,start,end):
    #cancel previous youtube
    cleanup()
    #download youtube
    #download_youtube(url) # with youtube-dl (slow)
    input_videos=download_video(url)
    # Get the current working directory
    parent_dir = os.getcwd()
    # Trim the video (start, end) seconds
    start =  start
    end =  end
    #Note: the trimmed video must have face on all frames
    #interval = end - start
    interval = time_between(start, end)
    #trimmed_video= parent_dir+'/sample_data/input_vid{}.mp4'.format(random.randint(0, 10000))
    #trimmed_audio= parent_dir+'/sample_data/input_audio{}.mp3'.format(random.randint(0, 10000))
    trimmed_video= parent_dir+'/sample_data/input_video.mp4'
    trimmed_audio= parent_dir+'/sample_data/input_audio.mp3'
    #delete trimmed if already exits
    clean_data()
    #call(["rm","-f",trimmed_audio])
    #call(["rm","-f",trimmed_video])
    
    #!rm -f {trimmed_video}
    # cut the video  
    call(["ffmpeg","-y","-i",input_videos,"-ss", start,"-t",interval,"-async","1",trimmed_video])
    #!ffmpeg -y -i youtube.mp4 -ss {start} -t {interval} -async 1 {trimmed_video}
    # cut the audio
    call(["ffmpeg","-i",trimmed_video, "-q:a", "0", "-map","a",trimmed_audio])
    #Preview trimmed video
    #clear_output()
    print("Trimmed Video+Audio")
    return trimmed_video, trimmed_audio

## We verify the pandas and numpy versions

In [41]:
print("In our enviroment")
os.system('{} pip show pandas numpy'.format(env))

In our enviroment
Name: pandas
Version: 1.3.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: gradio, TTS
---
Name: numpy
Version: 1.19.5
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages
Requires: 
Required-by: gradio, librosa, matplotlib, numba, opencv-contrib-python, opencv-python, pandas, resampy, scikit-learn, scipy, tensorboardX, torchvision, TTS, umap-learn


0

In [45]:
base='source activate python3 &&'
print("In the base enviroment")
os.system('{} pip show pandas numpy'.format(base))

In the base enviroment
Name: pandas
Version: 1.3.4
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: autovizwidget, hdijupyterutils, sagemaker, seaborn, shap, smclarify, sparkmagic, statsmodels
---
Name: numpy
Version: 1.20.3
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages
Requires: 
Required-by: astropy, bkcharts, bokeh, Bottleneck, daal4py, h5py, hdijupyterutils, imagecodecs, imageio, matplotlib, mkl-fft, mkl-random, numba, numexpr, opencv-python, pandas, patsy, pyarrow, pyerfa, PyWavelets, sagemaker, sag

0

In [46]:
print("In the Jupyter enviroment")
os.system('pip show pandas numpy')

In the Jupyter enviroment
Name: pandas
Version: 1.3.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: autovizwidget, hdijupyterutils, sagemaker, sparkmagic
---
Name: numpy
Version: 1.21.6
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages
Requires: 
Required-by: Bottleneck, hdijupyterutils, numexpr, pandas, sagemaker, sagemaker-pyspark, sparkmagic


0

In [47]:
import numpy
numpy.version.version

'1.19.5'

In [48]:
if is_first_time:
    os.system('{} pip install opencv-contrib-python-headless==4.1.2.30'.format(env))

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 70.1 MB/s eta 0:00:00


In [49]:
import sys
VOICE_PATH = "utils/"
# add libraries into environment
sys.path.append(VOICE_PATH) # set this if VOICE is not installed globally

In [50]:
from utils.voice import *

/home/ec2-user/SageMaker/VideoMessageGen
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:False
 | > do_amp_to_db_mel:True
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Using model: vits
 > Speaker manager is loaded with 6 speakers: female-en-5, female-en-5
, female-pt-4
, male-en-2, male-en-2
, male-pt-3

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None

In [51]:
def create_video(Text,Voicetoclone):
    out_audio=greet(Text,Voicetoclone)
    current_dir=os.getcwd()
    clonned_audio = os.path.join(current_dir, out_audio)
    
    #Start Crunching and Preview Output
    #Note: Only change these, if you have to
    pad_top =  0#@param {type:"integer"}
    pad_bottom =  10#@param {type:"integer"}
    pad_left =  0#@param {type:"integer"}
    pad_right =  0#@param {type:"integer"}
    rescaleFactor =  1#@param {type:"integer"}
    nosmooth = False #@param {type:"boolean"}
    
    out_name ="result_voice.mp4"
    out_file="../"+out_name
    
    if nosmooth == False:
        is_command_ok = os.system('{} cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "../sample_data/input_video.mp4" --audio "../out/clonned_audio.wav" --outfile {} --pads {} {} {} {} --resize_factor {}'.format(env,out_file,pad_top ,pad_bottom ,pad_left ,pad_right ,rescaleFactor))
    else:
        is_command_ok = os.system('{} cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "../sample_data/input_video.mp4" --audio "../out/clonned_audio.wav" --outfile {} --pads {} {} {} {} --resize_factor {} --nosmooth'.format(env,out_file,pad_top ,pad_bottom ,pad_left ,pad_right ,rescaleFactor))

    if is_command_ok > 0:
        print("Error : Ensure the video contains a face in all the frames.")
        out_file="./demo/tryagain1.mp4"
        return  out_file
    else:
        print("OK") 
    #clear_output()
    print("Creation of Video done")
    return out_name

In [52]:
URL = 'https://www.youtube.com/watch?v=xw5dvItD5zY'
#Testing first time
trimmed_video, trimmed_audio=youtube_trim(URL,"00:00:01","00:00:10")
showVideo(trimmed_video)

Deleting youtube1253.mp4
Deleting result_voice.mp4
Deleting youtube7136.mp4
Downloading...
Downloaded
Path to clean: /home/ec2-user/SageMaker/VideoMessageGen/sample_data/
Inside to clean /home/ec2-user/SageMaker/VideoMessageGen/sample_data
Restoring the path
Current directory is- /home/ec2-user/SageMaker/VideoMessageGen


ffmpeg version 5.1.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      57. 28.100 / 57. 28.100
  libavcodec     59. 37.100 / 59. 37.100
  libavformat   

Trimmed Video+Audio


size=     165kB time=00:00:09.01 bitrate= 150.2kbits/s speed=76.7x    
video:0kB audio:165kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.210846%


In [53]:
Text=' I am clonning your voice. Charles!. Machine intelligence is the last invention that humanity will ever need to make.'
Voicetoclone=trimmed_audio
print(Voicetoclone)
#Testing first time
outvideo=create_video(Text,Voicetoclone)
#Preview output video
print("Final Video Preview")
final_video= parent_dir+'/'+outvideo
print("Dowload this video from", final_video)
showVideo(final_video)

usage: ffmpeg-normalize [-h] [-o OUTPUT [OUTPUT ...]] [-of OUTPUT_FOLDER] [-f]
                        [-d] [-v] [-q] [-n] [-pr] [--version]
                        [-nt {ebu,rms,peak}] [-t TARGET_LEVEL] [-p]
                        [-lrt LOUDNESS_RANGE_TARGET]
                        [--keep-loudness-range-target] [-tp TRUE_PEAK]
                        [--offset OFFSET] [--dual-mono] [--dynamic]
                        [-c:a AUDIO_CODEC] [-b:a AUDIO_BITRATE]
                        [-ar SAMPLE_RATE] [-koa] [-prf PRE_FILTER]
                        [-pof POST_FILTER] [-vn] [-c:v VIDEO_CODEC] [-sn]
                        [-mn] [-cn] [-ei EXTRA_INPUT_OPTIONS]
                        [-e EXTRA_OUTPUT_OPTIONS] [-ofmt OUTPUT_FORMAT]
                        [-ext EXTENSION]
                        input [input ...]
ffmpeg-normalize: error: argument -o/--output: expected at least one argument
/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/librosa/core/audio.py:162: U

/home/ec2-user/SageMaker/VideoMessageGen/sample_data/input_audio.mp3
path url
/home/ec2-user/SageMaker/VideoMessageGen/sample_data/input_audio.mp3
 > text:  I am clonning your voice. Charles!. Machine intelligence is the last invention that humanity will ever need to make.
Generated Audio


 > Saving output to out/clonned_audio.wav


  0%|          | 0/12 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)

100%|██████████| 2/2 [02:02<00:00, 61.34s/it]ffmpeg version 5.1.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 225
(80, 605)
Length of mel chunks: 186
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded
OK
Creation of Video done
Final Video Preview
Dowload this video from /home/ec2-user/SageMaker/VideoMessageGen/result_voice.mp4


In [54]:
parent_dir

'/home/ec2-user/SageMaker/VideoMessageGen'

In [55]:
def time_format_check(input1):
    timeformat = "%H:%M:%S"
    #input1 = input("At what time did sensor 1 actuate? ")
    try:
        validtime = datetime.strptime(input1, timeformat)
        print("The time format is valid", input1)
        #Do your logic with validtime, which is a valid format
        return False
    except ValueError:
        print("The time {}  has not valid format hh:mm:ss".format(input1))
        return True

In [56]:
def to_seconds(datetime_obj):
    from datetime import datetime
    time =datetime_obj
    date_time = datetime.strptime(time, "%H:%M:%S")
    a_timedelta = date_time - datetime(1900, 1, 1)
    seconds = a_timedelta.total_seconds()
    return seconds

In [57]:
def validate_youtube(url):
    #This creates a youtube objet
    try:
        yt = YouTube(url)  
    except Exception:
        print("Hi there URL seems invalid")
        return True, 0
    #This will return the length of the video in sec as an int
    video_length = yt.length
    if    video_length > 600:
        print("Your video is larger than 10 minutes")
        return True, video_length
    else:
        print("Your video is less than 10 minutes")
        return False, video_length

In [58]:
deltas=time_between("00:06:00","00:07:00" )
len(str(deltas))

7

In [59]:
def video_generator(text_to_say,url,initial_time,final_time):
    print('Checking the url',url)
    check1, video_length = validate_youtube(url)
    if check1 is True: return "./demo/tryagain2.mp4"
    check2 = validate_time(initial_time,final_time, video_length)
    if check2 is True: return "./demo/tryagain0.mp4"
    trimmed_video, trimmed_audio=youtube_trim(url,initial_time,final_time)
    voicetoclone=trimmed_audio
    print(voicetoclone)
    outvideo=create_video(text_to_say,voicetoclone)
    #Preview output video
    print("Final Video Preview")
    final_video= parent_dir+'/'+outvideo
    print("DONE")
    #showVideo(final_video)
    return final_video

In [60]:
def validate_time(initial_time,final_time,video_length):
    is_wrong1=time_format_check(initial_time)
    is_wrong2=time_format_check(final_time) 
    #print(is_wrong1,is_wrong2)
    if is_wrong1 is False and is_wrong2 is False:
        delta=time_between(initial_time,final_time)
        if len(str(delta)) > 8:
            print("Final Time is Smaller than Initial Time: t1>t2")
            is_wrong = True
            return is_wrong
        else:
            print("OK")
            is_wrong=False
            if int(to_seconds(delta)) > 300 :
                print("The trim is larger than 5 minutes")
                is_wrong = True
                return is_wrong
            
            elif int(to_seconds(delta)) > video_length :
                print("The trim is larger than video lenght")
                is_wrong = True
                return is_wrong             
            else:
                return  is_wrong
            
    else:
        print("Your time format is invalid")
        is_wrong = True
        return is_wrong

In [61]:
validate_time("00:00:01","00:05:00", 200)

The time format is valid 00:00:01
The time format is valid 00:05:00
OK
The trim is larger than video lenght


True

## Unit tests

In [62]:
URL = 'https://www.youtube.com/watch?v=xw5dvItD5zY'
Text=' I am clonning your voice. Charles!. Machine intelligence is the last invention that humanity will ever need to make.'
# No issue
#video_generator(Text,URL,"00:00:01","00:00:10")

In [63]:
validate_youtube(URL)

Your video is less than 10 minutes


(False, 47)

In [64]:
# No issue
#video_generator(Text,URL,"00:00:01","00:00:10")

In [65]:
# Initial time > Final time
video_generator(Text, URL,"00:00:10","00:00:01")

Checking the url https://www.youtube.com/watch?v=xw5dvItD5zY
Your video is less than 10 minutes
The time format is valid 00:00:10
The time format is valid 00:00:01
Final Time is Smaller than Initial Time: t1>t2


'./demo/tryagain0.mp4'

In [66]:
#Trim is larger than video
video_generator(Text, URL,"00:00:01","00:05:00")

Checking the url https://www.youtube.com/watch?v=xw5dvItD5zY
Your video is less than 10 minutes
The time format is valid 00:00:01
The time format is valid 00:05:00
OK
The trim is larger than video lenght


'./demo/tryagain0.mp4'

In [67]:
#Trim is larger than limit of 4 min
video_generator(Text, URL,"00:00:01","00:06:00")

Checking the url https://www.youtube.com/watch?v=xw5dvItD5zY
Your video is less than 10 minutes
The time format is valid 00:00:01
The time format is valid 00:06:00
OK
The trim is larger than 5 minutes


'./demo/tryagain0.mp4'

# Web Appp Code

In [77]:
text_to_say=gr.inputs.Textbox(label='What would you like the voice to say? (max. 2000 characters per request)')
url =gr.inputs.Textbox(label = "Enter the YouTube URL below:")
initial_time = gr.inputs.Textbox(label='Initial time of trim? (format: hh:mm:ss)')
final_time= gr.inputs.Textbox(label='Final time to trim? (format: hh:mm:ss)')
demo=gr.Interface(fn = video_generator,
            inputs = [text_to_say,url,initial_time,final_time],
            outputs = 'video', 
            verbose = True,
            title = 'Video Speech Generator from Youtube Videos',
            description = 'A simple application that replaces the original speech of the video by your text. Wait one minute to process.',
            article = 
                        '''<div>
                            <p style="text-align: center"> 
                            All you need to do is to paste the Youtube link and 
                            set the initial time and final time of the real speach.
                            (The limit of the trim is 5 minutes and not larger than video length)
                            hit submit, then wait for compiling.
                            After that click on Play/Pause for listing to the video. 
                            The video is saved in an mp4 format. Ensure the video contains a face in all the frames.
                             For more information visit <a href="https://ruslanmv.com/">ruslanmv.com</a>
                            </p>
                        </div>''',
           enable_queue=True,
           examples = [['I am clonning your voice. Charles!. Machine intelligence is the last invention that humanity will ever need to make.',
                        "https://www.youtube.com/watch?v=xw5dvItD5zY", 
                        "00:00:01","00:00:10"],
                        ['I am clonning your voice. Jim Carrey!. Machine intelligence is the last invention that humanity will ever need to make.',
                        "https://www.youtube.com/watch?v=uIaY0l5qV0c",
                        "00:00:29",  "00:01:05"],
                        ['I am clonning your voice. Mark Zuckerberg!. Machine intelligence is the last invention that humanity will ever need to make.',
                        "https://www.youtube.com/watch?v=AYjDIFrY9rc",
                        "00:00:11", "00:00:44"],
                        ['I am clonning your voice. Ronald Reagan!. Machine intelligence is the last invention that humanity will ever need to make.',
                        "https://www.youtube.com/watch?v=iuoRDY9c5SQ",
                        "00:01:03",  "00:01:22"], 
                        ['I am clonning your voice. Elon Musk!. Machine intelligence is the last invention that humanity will ever need to make.',
                        "https://www.youtube.com/watch?v=IZ8JQ_1gytg",
                        "00:00:10",  "00:00:43"],
                        ['I am clonning your voice. Hitler!. Machine intelligence is the last invention that humanity will ever need to make.',
                        "https://www.youtube.com/watch?v=F08wrLyH5cs",
                        "00:00:15",  "00:00:40"],
                         ['I am clonning your voice. Alexandria!. Machine intelligence is the last invention that humanity will ever need to make.',
                        "https://www.youtube.com/watch?v=Eht6oIkzkew",
                        "00:00:02",  "00:00:30"], 
                        ],
                    allow_flagging=False
            )

/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/gradio/deprecation.py:40: UserWarning: `verbose` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/gradio/deprecation.py:40: UserWarning: `enable_queue` is deprecated in `Interface()`, please use it within `launch()` instead.
  warnings.warn(value)
/h

In [80]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7862/
Running on public URL: https://18966.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f975aab29d0>,
 'http://127.0.0.1:7862/',
 'https://18966.gradio.app')

Checking the url https://www.youtube.com/watch?v=AYjDIFrY9rc
Your video is less than 10 minutes
The time format is valid 00:00:11
The time format is valid 00:00:44
OK
Downloading...
Downloaded
Path to clean: /home/ec2-user/SageMaker/VideoMessageGen/sample_data/
Inside to clean /home/ec2-user/SageMaker/VideoMessageGen/sample_data
Deleting input_video.mp4
Deleting input_audio.mp3
Restoring the path
Current directory is- /home/ec2-user/SageMaker/VideoMessageGen


ffmpeg version 5.1.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      57. 28.100 / 57. 28.100
  libavcodec     59. 37.100 / 59. 37.100
  libavformat   

Trimmed Video+Audio
/home/ec2-user/SageMaker/VideoMessageGen/sample_data/input_audio.mp3
path url
/home/ec2-user/SageMaker/VideoMessageGen/sample_data/input_audio.mp3
 > text: I am clonning your voice. Mark Zuckerberg!. Machine intelligence is the last invention that humanity will ever need to make.
Generated Audio


 > Saving output to out/clonned_audio.wav


  0%|          | 0/10 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)

100%|██████████| 2/2 [00:19<00:00,  9.91s/it]ffmpeg version 5.1.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 990
(80, 436)
Length of mel chunks: 159
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded
OK
Creation of Video done
Final Video Preview
DONE


In [79]:
demo.close()

Closing server running on port: 7862


In [70]:
# No issues tests
is_first_time =False

In [71]:
if is_first_time:
    URL = "https://www.youtube.com/watch?v=iuoRDY9c5SQ"
    Text=' I am clonning your voice. Charles!. Machine intelligence is the last invention that humanity will ever need to make.'
    video_generator(Text,URL,  "00:01:04",  "00:01:21")
    #Preview output video
    print("Final Video Preview")
    final_video= parent_dir+'/Wav2Lip/results/result_voice.mp4'
    print("Dowload this video from", final_video)
    showVideo(final_video)

In [81]:
clean_data()
cleanup()

Path to clean: /home/ec2-user/SageMaker/VideoMessageGen/sample_data/
Inside to clean /home/ec2-user/SageMaker/VideoMessageGen/sample_data
Deleting input_video.mp4
Deleting input_audio.mp3
Restoring the path
Current directory is- /home/ec2-user/SageMaker/VideoMessageGen
Deleting youtube8288.mp4
Deleting result_voice.mp4


In [82]:
#os.system('{}pip list --format=freeze > requirements_sagemaker.txt'.format(env))